In [147]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import sparse

In [133]:
users_ratings = pd.read_csv('../ratings.csv')

In [134]:
user_ratings = users_ratings.drop('timestamp',axis=1)
print(user_ratings.shape)

(100836, 3)


In [135]:
user_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [36]:
print(user_ratings['userId'].unique().shape[0])
user_ratings['movieId'].unique().shape[0]

610


9724

Дана матрица предпочтений User/Item. Сформировать рекомендации с
помощью SVD.
1.1. Предсказать оценки в выделенных ячейках.

Решение:
1. Создаём pivot_table
2. Заменяем NaN на 0 в pivot_table
3. Вычисляем предикты
4. Заменяем нули предиктами
5. Применяем SVD
6. Определяем размерность матрицы содержащий 90% информации
7. Получаем таблицу с предсказанными оценками
8. Получаем проноз

In [50]:
user_movie_rating = users_ratings.pivot_table(index='userId', columns='movieId', values='rating')
user_movie_rating

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
user_movie_rating.fillna(0.0, inplace=True)
user_movie_rating

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
## Вычисление предиктов
global_mean = user_movie_rating[user_movie_rating>0].mean().mean()
print(global_mean)
user_movie_rating_minus_mean = user_movie_rating[user_movie_rating>0]-global_mean
user_bias = user_movie_rating_minus_mean.T.mean()
print(user_bias)
item_bias = user_movie_rating_minus_mean.apply(lambda r: r - user_bias).mean()
print(item_bias)

3.2624482748109656
userId
1      1.103931
2      0.685828
3     -0.826551
4      0.293107
5      0.373915
         ...   
606    0.394951
607    0.523648
608   -0.128273
609    0.007822
610    0.426108
Length: 610, dtype: float64
movieId
1         0.312167
2        -0.053158
3        -0.234798
4        -1.096045
5        -0.522626
            ...   
193581    0.294776
193583   -0.205224
193585   -0.205224
193587   -0.205224
193609    0.372024
Length: 9724, dtype: float64


In [53]:
# заменяем нули предиктами
for rowIndex, row in tqdm(user_movie_rating.iterrows()): #iterate over rows
    for columnIndex, value in row.items():
        if (value == 0):
            user_movie_rating.loc[rowIndex][columnIndex]=global_mean +user_bias[rowIndex]+item_bias[columnIndex]
print(user_movie_rating)

610it [06:03,  1.68it/s]

movieId    1         2         3         4         5         6         7       \
userId                                                                          
1        4.000000  4.313221  4.000000  3.270335  3.843753  4.000000  3.965652   
2        4.260443  3.895118  3.713478  2.852231  3.425650  4.326737  3.547548   
3        2.748064  2.382739  2.201100  1.339853  1.913271  2.814358  2.035170   
4        3.867722  3.502397  3.320758  2.459511  3.032930  3.934016  3.154828   
5        4.000000  3.583205  3.401566  2.540319  3.113738  4.014824  3.235636   
...           ...       ...       ...       ...       ...       ...       ...   
606      2.500000  3.604241  3.422601  2.561354  3.134773  4.035860  2.500000   
607      4.000000  3.732938  3.551299  2.690052  3.263470  4.164557  3.385369   
608      2.500000  2.000000  2.000000  2.038131  2.611550  3.512636  2.733448   
609      3.000000  3.217112  3.035473  2.174226  2.747644  3.648731  2.869543   
610      5.000000  3.635398 

In [54]:
from numpy import linalg
U, Sigma, Vt = linalg.svd(user_movie_rating)

In [55]:
# print(Sigma)
print(Sigma.sum())
# 90%
print(Sigma.sum()*0.9)
print(Sigma[:300].sum())

14032.238225097439
12629.014402587694
12890.103191904567


In [56]:
k=300

In [71]:
def rank_k2(k):
    U_reduced= np.mat(U[:,:k])
    Vt_reduced = np.mat(Vt[:k,:])
    Sigma_reduced = Sigma_reduced = np.eye(k)*Sigma[:k]
    Sigma_sqrt = np.sqrt(Sigma_reduced)
    return U_reduced*Sigma_sqrt, Sigma_sqrt*Vt_reduced,Sigma_reduced
U_reduced, Vt_reduced,Sigma_reduced = rank_k2(4)
predicted_user_movie_rating = U_reduced * Vt_reduced
print(predicted_user_movie_rating)

[[4.63386331 4.31456242 4.16118239 ... 4.16204079 4.16204079 4.73896909]
 [4.23666034 3.89693703 3.71692381 ... 3.74287318 3.74287318 4.3201915 ]
 [2.78304957 2.38122351 2.14667357 ... 2.22885734 2.22885734 2.80580186]
 ...
 [3.63328316 3.0963976  2.82321965 ... 2.93381557 2.93381557 3.51592362]
 [3.5920589  3.21688039 3.02048589 ... 3.06513224 3.06513224 3.64188958]
 [4.29510655 3.6066629  3.55410678 ... 3.4808043  3.4808043  4.06058414]]


In [72]:
# оценка пользователя 1 для фильма 47
predicted_user_movie_rating[47,1]

3.9782898770532205

1.2. Добавить пользователя Студента группы и предсказать оценки для него.
1. Создаём вектор пользователя
2. Рассчитываем вменение
3. Получаем прогноз

In [73]:
new_user = np.zeros(predicted_user_movie_rating.shape[1])
print(new_user.shape)
# случайные оценки
new_user[510] = 5.0
new_user[320] = 3.0
new_user[11] = 5.0

(9724,)


In [75]:
# делаем вменение
from numpy.linalg import inv
new_user_average = new_user[new_user > 0.0].mean()
print(new_user_average)
new_user[new_user == 0] = np.NaN
np.nan_to_num(new_user,copy=False,nan=new_user_average)
print(new_user)

new_user = new_user *Vt_reduced.T* inv(Sigma_reduced)
print(new_user)

4.333333333333334
[4.33333333 4.33333333 4.33333333 ... 4.33333333 4.33333333 4.33333333]
[[-4.45400155 -5.5948098  -0.52094966 -0.05899722]]


In [76]:
## Прогноз для нового пользователя
movie = 1
print(new_user*Vt_reduced[:,movie])

[[4.34581336]]


## Здание 2
Дана матрица предпочтений User/Item. Добавить строку с оценками
Студента группы. Определить схожесть пользователей и Студента группы.
2.1. Косинусное расстояние
2.2. Коэффициент Пирсона
2.3. Коэффициент Отиаи

In [136]:
# Добавляем нового пользователя
columns = ['userId', 'movieId', 'rating']
new_user_ratings = pd.DataFrame([
    [611,11,5.0],
    [611,12,3.0],
    [611,417,4.0],
],columns=columns)

merged_user_ratings = pd.concat([user_ratings,new_user_ratings],ignore_index=True,)
merged_user_ratings.tail(7)

,userId,movieId,rating
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0
100835,610,170875,3.0
100836,611,11,5.0
100837,611,12,3.0
100838,611,417,4.0


In [137]:
# Нормализуем оценки вычтя из оценки среднюю оценку пользователя
count_col = merged_user_ratings.groupby(['userId'])['rating'].count()
std_col = merged_user_ratings.groupby(['userId'])['rating'].std()
mean_col = merged_user_ratings.groupby(['userId'])['rating'].mean()
max_col = merged_user_ratings.groupby(['userId'])['rating'].max()
min_col = merged_user_ratings.groupby(['userId'])['rating'].min()

def rowFunc(row):
    return 0.0 if (count_col.at[row['userId']]==1) or (std_col.at[row['userId']]==0)  else(row['rating']-mean_col.at[row['userId']])/(max_col.at[row['userId']]-min_col.at[row['userId']])

merged_user_ratings['mean'] = merged_user_ratings.apply(rowFunc,axis=1)


# for index, row in merged_user_ratings.iterrows():
#     merged_user_ratings.at[index,'mean']= 0.0 if (count_col.at[row['userId']]==1) or (std_col.at[row['userId']]==0)  else(row['rating']-mean_col.at[row['userId']])/(max_col.at[row['userId']]-min_col.at[row['userId']])
    
merged_user_ratings.shape

(100839, 4)

In [138]:
# проверяем, что нет пропусков
merged_user_ratings['mean'].isnull().any()

False

In [139]:
tqdm.pandas()

In [140]:
users_values = merged_user_ratings['userId'].sort_values().unique()
movies_values = merged_user_ratings['movieId'].sort_values().unique()

user_categories = dict(enumerate(users_values))
movie_categories = dict(enumerate(movies_values))

def rowFunc(row):
#     print(np.where(movies_values == row['movieId'])[0], np.where(users_values == row['userId'])[0])
    return np.where(movies_values == row['movieId'])[0][0], np.where(users_values == row['userId'])[0][0]

merged_user_ratings[['movieId','userId']] = merged_user_ratings.progress_apply(rowFunc,axis=1,result_type="expand")

# merged_user_ratings['movieId'],merged_user_ratings['userId'] = tqdm(zip(*merged_user_ratings.apply(rowFunc,axis=1)))
# for index, row in tqdm(merged_user_ratings.iterrows()):
#     merged_user_ratings.at[index,'movieId'] = np.where(movies_values == row['movieId'])[0]
#     merged_user_ratings.at[index,'userId'] = np.where(users_values == row['userId'])[0]
merged_user_ratings

100%|███████████████████████████████████████████████████████████████████████| 100839/100839 [00:04<00:00, 20610.79it/s]


,userId,movieId,rating,mean
0,0,0,4.0,-0.091595
1,0,2,4.0,-0.091595
2,0,5,4.0,-0.091595
3,0,43,5.0,0.158405
4,0,46,5.0,0.158405
...,...,...,...,...
100834,609,9445,5.0,0.291432
100835,609,9485,3.0,-0.153012
100836,610,10,5.0,0.500000
100837,610,11,3.0,-0.500000


In [141]:
from scipy.sparse import coo_matrix
coo = coo_matrix((merged_user_ratings['mean'], (merged_user_ratings['userId'],merged_user_ratings['movieId'])))

In [142]:
coo.shape

(611, 9724)

In [143]:
csr = coo.tocsr()
csr.shape

(611, 9724)

In [144]:
# a = csr.getrow(0)
# b = csr.getrow(1)
# print(a.toarray())
# print(b)
# np.sum(a*b)
# np.sum(a*b)/(np.sqrt(np.sum(np.square(a)))*np.sqrt(np.sum(np.square(b))))

In [145]:
# for i in range(csr.shape[0]): 
#     for j in range(i,csr.shape[0]): 
#     pt = slice(csr.indptr[i], csr.indptr[i+1]) 
#     print(i, csr.indices[pt], csr.data[pt])

In [148]:
M=coo.toarray()
print(M.shape)

def calculate_cosine_distance_cor(M):
    Res = np.empty((M.shape[0],M.shape[0]))
    def my_cosine_distance(i,j):
        return np.sum(M[i,:]*M[j,:])/(np.sqrt(np.sum(np.square(M[i,:])))*np.sqrt(np.sum(np.square(M[j,:]))))
    for i in tqdm(range(M.shape[0])):
        for j in range(i,M.shape[0]):
            calculated = my_cosine_distance(i,j);
            if (i==j):
                Res[i,j]=calculated
            else:
                Res[i,j]=Res[j,i]=calculated
    Res[np.isnan(Res)] = 0
    return sparse.csr_matrix(Res)

cor = calculate_cosine_distance_cor(M)
print(cor.toarray())

  0%|                                                                                          | 0/611 [00:00<?, ?it/s]<ipython-input-148-ed2f5ed3bbae>:7: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(M[i,:]*M[j,:])/(np.sqrt(np.sum(np.square(M[i,:])))*np.sqrt(np.sum(np.square(M[j,:]))))
  1%|▊                                                                                 | 6/611 [00:00<00:11, 53.90it/s]

(611, 9724)


100%|███████████████████████████████████████████████████████████████████████████████| 611/611 [00:05<00:00, 111.22it/s]

[[ 1.00000000e+00  1.26451574e-03  5.52577176e-04 ... -2.57125541e-02
   1.09323166e-02  0.00000000e+00]
 [ 1.26451574e-03  1.00000000e+00  0.00000000e+00 ... -6.00909967e-02
   2.49992083e-02  0.00000000e+00]
 [ 5.52577176e-04  0.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   1.95499646e-02  0.00000000e+00]
 ...
 [-2.57125541e-02 -6.00909967e-02  0.00000000e+00 ...  1.00000000e+00
  -1.24714266e-02  0.00000000e+00]
 [ 1.09323166e-02  2.49992083e-02  1.95499646e-02 ... -1.24714266e-02
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]


In [149]:
cor.shape

(611, 611)

In [151]:
# Схожесть нового пользователя с другими косинусное расстояние
cor.toarray()[-1]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.02379198,  0.        ,  0.04562739,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.04626138,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.09756299,  0.        , -0.01456328,
        0.        ,  0.        ,  0.13644064,  0.        ,  0.        ,
        0.        ,  0.04552836, -0.05409425,  0.20976052, -0.03179004,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,         nan,  0.        ,  0.        ,
        0.02869854, -0.01187171,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.02814049,  0.        ,  0.  

In [153]:
cor.toarray()[-1][np.where(cor.toarray()[-1]>0.2)]

array([0.20976052, 0.2941742 , 1.        ])

In [154]:
M=coo.toarray()


def calculate_otiai_distance_cor(M):
    Res = np.empty((M.shape[0],M.shape[0]))
    def my_otiai_distance(i,j):
        i_mean = np.sum(M[i,:])/(M[i,:]!=0).sum()
 
        j_mean = np.sum(M[j,:])/(M[j,:]!=0).sum() 

        i_minus_i_mean =  np.where((M[i,:]!=0), M[i,:]-i_mean, 0)
     
        j_minus_j_mean =  np.where((M[j,:]!=0), M[j,:]-j_mean, 0)
  
        numerator = np.sum(i_minus_i_mean*j_minus_j_mean)

        denumerator = (np.sqrt(np.sum(np.square(i_minus_i_mean)))*np.sqrt(np.sum(np.square(j_minus_j_mean))))
        return numerator/denumerator
    for i in tqdm(range(M.shape[0])):
        for j in range(i,M.shape[0]):
            calculated = my_otiai_distance(i,j);
            if (i==j):
                Res[i,j]=calculated
            else:
                Res[i,j]=Res[j,i]=calculated
    Res[np.isnan(Res)] = 0
    return sparse.csr_matrix(Res)

cor = calculate_otiai_distance_cor(M)
print(cor.toarray())

  0%|                                                                                          | 0/611 [00:00<?, ?it/s]<ipython-input-154-dda248356c8d>:9: RuntimeWarning: invalid value encountered in double_scalars
  j_mean = np.sum(M[j,:])/(M[j,:]!=0).sum()
<ipython-input-154-dda248356c8d>:18: RuntimeWarning: invalid value encountered in double_scalars
  return numerator/denumerator
  9%|██████▉                                                                          | 52/611 [00:02<00:29, 19.00it/s]<ipython-input-154-dda248356c8d>:7: RuntimeWarning: invalid value encountered in double_scalars
  i_mean = np.sum(M[i,:])/(M[i,:]!=0).sum()
100%|████████████████████████████████████████████████████████████████████████████████| 611/611 [00:17<00:00, 34.91it/s]

[[ 1.00000000e+00  1.26451574e-03  5.52577176e-04 ... -2.57125541e-02
   1.09323166e-02  0.00000000e+00]
 [ 1.26451574e-03  1.00000000e+00  0.00000000e+00 ... -6.00909967e-02
   2.49992083e-02  0.00000000e+00]
 [ 5.52577176e-04  0.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   1.95499646e-02  0.00000000e+00]
 ...
 [-2.57125541e-02 -6.00909967e-02  0.00000000e+00 ...  1.00000000e+00
  -1.24714266e-02  0.00000000e+00]
 [ 1.09323166e-02  2.49992083e-02  1.95499646e-02 ... -1.24714266e-02
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]


In [155]:
# Схожесть нового пользователя с другими Отиаи
cor.toarray()[-1][np.where(cor.toarray()[-1]>0.2)]

array([0.20976052, 0.2941742 , 1.        ])

In [156]:
M=coo.toarray()
print(M.shape)

def calculate_pearson_distance_cor(M):
    Res = np.empty((M.shape[0],M.shape[0]))
    def my_pearson_distance(i,j):
        a = np.sum(M[i,:])/(M[i,:]!=0).sum()
        b = np.sum(M[j,:])/(M[j,:]!=0).sum()
        aa = np.where((M[i,:]!=0) &(M[j,:]!=0), M[i,:]-a, 0)
        bb = np.where((M[i,:]!=0) &(M[j,:]!=0), M[j,:]-b, 0)
        c = np.sum(aa*bb)
        d = (np.sqrt(np.sum(np.square(aa)))*np.sqrt(np.sum(np.square(bb))))
   
        return c/d
    for i in tqdm(range(M.shape[0])):
        for j in range(i,M.shape[0]):
            calculated = my_pearson_distance(i,j);
            if (i==j):
                Res[i,j]=calculated
            else:
                Res[i,j]=Res[j,i]=calculated
    Res[np.isnan(Res)] = 0
    return sparse.csr_matrix(Res)

cor = calculate_pearson_distance_cor(M)
print(cor.toarray())

  0%|                                                                                          | 0/611 [00:00<?, ?it/s]<ipython-input-156-b7b47c3406b9>:8: RuntimeWarning: invalid value encountered in double_scalars
  b = np.sum(M[j,:])/(M[j,:]!=0).sum()
<ipython-input-156-b7b47c3406b9>:14: RuntimeWarning: invalid value encountered in double_scalars
  return c/d
  0%|▎                                                                                 | 2/611 [00:00<00:40, 14.97it/s]

(611, 9724)


  9%|██████▉                                                                          | 52/611 [00:03<00:31, 17.64it/s]<ipython-input-156-b7b47c3406b9>:7: RuntimeWarning: invalid value encountered in double_scalars
  a = np.sum(M[i,:])/(M[i,:]!=0).sum()
100%|████████████████████████████████████████████████████████████████████████████████| 611/611 [00:18<00:00, 32.67it/s]

[[ 1.          1.          0.01122517 ... -0.28680855  0.0968699
   0.        ]
 [ 1.          1.          0.         ... -1.          0.32000431
   0.        ]
 [ 0.01122517  0.          1.         ...  0.          0.59588158
   0.        ]
 ...
 [-0.28680855 -1.          0.         ...  1.         -0.20662762
   0.        ]
 [ 0.0968699   0.32000431  0.59588158 ... -0.20662762  1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   1.        ]]


In [157]:
# Схожесть нового пользователя с другими Пирсон
cor.toarray()[-1][np.where(cor.toarray()[-1]>0.2)]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.61442114, 1.        ])

## Задание 5
Дана матрица предпочтений User/Item. Построить матрицу схожести, с
помощью асимметричного коэффициента Жаккара и методом ассоциаций. 

In [158]:
M=coo.toarray()

def calculate_jaccard_asim_distance_cor(M):
    Res = np.empty((M.shape[0],M.shape[0]))
    def my_jaccard_asim_distance(i,j):
        and_jack = np.where((M[i,:]!=0) & (M[j,:]!=0), 1, 0).sum() 
        or_jack = np.where((M[i,:]!=0), 1, 0).sum()

   
        return and_jack/or_jack
    for i in tqdm(range(M.shape[0])):
        for j in range(M.shape[0]):
            calculated = my_jaccard_asim_distance(i,j);
            Res[i,j]=calculated
    Res[np.isnan(Res)] = 0
    return sparse.csr_matrix(Res)

cor = calculate_jaccard_asim_distance_cor(M)
print(cor.toarray())

  8%|██████▎                                                                          | 48/611 [00:00<00:11, 49.56it/s]<ipython-input-158-b946448946a0>:10: RuntimeWarning: invalid value encountered in long_scalars
  return and_jack/or_jack
100%|████████████████████████████████████████████████████████████████████████████████| 611/611 [00:12<00:00, 47.71it/s]

[[1.         0.00862069 0.03017241 ... 0.0387931  0.29741379 0.        ]
 [0.06896552 1.         0.         ... 0.03448276 0.62068966 0.        ]
 [0.17948718 0.         1.         ... 0.         0.17948718 0.        ]
 ...
 [0.24324324 0.02702703 0.         ... 1.         0.27027027 0.        ]
 [0.05299539 0.01382488 0.00537634 ... 0.00768049 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [159]:
M=coo.toarray()

def calculate_associations_distance_cor(M):
    Res = np.empty((M.shape[0],M.shape[0]))
    def my_associations_distance(i,j):
        and_i_j = np.where((M[i,:]!=0) & (M[j,:]!=0), 1, 0).sum() 
        and_not_i_j = np.where((M[i,:]==0) & (M[j,:]!=0), 1, 0).sum() 
        
   
        return and_i_j/and_not_i_j
    for i in tqdm(range(M.shape[0])):
        for j in range(M.shape[0]):
            calculated = my_associations_distance(i,j);
#             if (i==j):
#                 Res[i,j]=calculated
#             else:
            Res[i,j]=calculated
    Res[np.isnan(Res)] = 0
    Res[np.isinf(Res)] = 0
    return sparse.csr_matrix(Res)

cor = calculate_associations_distance_cor(M)
print(cor.toarray())

  0%|                                                                                          | 0/611 [00:00<?, ?it/s]<ipython-input-159-03a38c7d6568>:10: RuntimeWarning: divide by zero encountered in long_scalars
  return and_i_j/and_not_i_j
<ipython-input-159-03a38c7d6568>:10: RuntimeWarning: invalid value encountered in long_scalars
  return and_i_j/and_not_i_j
100%|████████████████████████████████████████████████████████████████████████████████| 611/611 [00:13<00:00, 44.96it/s]

[[0.         0.07407407 0.21875    ... 0.32142857 0.05596107 0.        ]
 [0.00869565 0.         0.         ... 0.02777778 0.01401869 0.        ]
 [0.03111111 0.         0.         ... 0.         0.00540541 0.        ]
 ...
 [0.04035874 0.03571429 0.         ... 0.         0.00773994 0.        ]
 [0.42331288 1.63636364 0.21875    ... 0.37037037 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [173]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import KFold
from surprise import SlopeOne
from surprise import accuracy

## Задание 3
По расширенной матрице из задания 2. с помощью алгоритма
кластеризации k ближайших соседей предсказать оценки

In [172]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_user_ratings[["userId", "movieId", "rating"]], reader)
trainset = data.build_full_trainset()


In [174]:
#Use KNNBasic algorithm
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [175]:
uid =str(200)
iid = str(100)
pred = algo.predict(uid,iid)
pred.est

3.5015718124931823

In [176]:
users_ratings_prediction = pd.DataFrame([],columns=['userId','movieId','rating'])
users = pd.Series(pd.unique(merged_user_ratings['userId']))
for _,user in tqdm(users.iteritems()):
    not_user_movies = pd.Series(pd.unique(merged_user_ratings.loc[merged_user_ratings['userId']!=user,'movieId']))
    user_predictions = not_user_movies.apply(lambda movie: algo.predict(str(user),str(movie)).est)
    df = pd.DataFrame(columns = ['movieId', 'rating'])
    df['movieId']=not_user_movies
    df['rating']=user_predictions
    df['userId']=user
    users_ratings_prediction = pd.concat([users_ratings_prediction, df],ignore_index=True)

611it [00:48, 12.56it/s]


In [177]:
users_ratings_prediction

,userId,movieId,rating
0,0,277,3.501572
1,0,291,3.501572
2,0,1283,3.501572
3,0,2670,3.501572
4,0,4607,3.501572
...,...,...,...
5937913,610,9307,3.501572
5937914,610,9312,3.501572
5937915,610,9324,3.501572
5937916,610,9371,3.501572


## Задание 4
Дана матрица предпочтений User/Item. Добавить строку с оценками
Студента группы. Предсказать оценку для выделенных ячеек с помощью
алгоритма Slope One. 

In [161]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_user_ratings[["userId", "movieId", "rating"]], reader)
trainset = data.build_full_trainset()

In [162]:
#Use SlopeOne algorithm
algo = SlopeOne()
algo.fit(trainset)

In [165]:
uid =str(200)
iid = str(100)
pred = algo.predict(uid,iid)
pred.est

3.5015718124931823

In [169]:
user_ratings_prediction = pd.DataFrame([],columns=['userId','movieId','rating'])
user = 611
not_user_movies = pd.Series(pd.unique(merged_user_ratings.loc[merged_user_ratings['userId']!=user,'movieId']))
user_predictions = not_user_movies.apply(lambda movie: algo.predict(str(user),str(movie)).est)
df = pd.DataFrame(columns = ['movieId', 'rating'])
df['movieId']=not_user_movies
df['rating']=user_predictions
df['userId']=user
user_ratings_prediction = pd.concat([user_ratings_prediction, df],ignore_index=True)

In [170]:
user_ratings_prediction

,userId,movieId,rating
0,611,0,3.501572
1,611,2,3.501572
2,611,5,3.501572
3,611,43,3.501572
4,611,46,3.501572
...,...,...,...
9719,611,9307,3.501572
9720,611,9312,3.501572
9721,611,9324,3.501572
9722,611,9371,3.501572
